In [18]:
# Required libraries
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [30]:
PINECONE_API_KEY = 'xxxxxxxxxxxxxxxxxxxxxx'
PINECONE_API_ENV = 'xxxxxxxxxxxxxx' #using https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 for embedding this model consists as 384 dim

In [20]:
#Extracting Geotechnical data from pdf from data folder
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [21]:
extracted_data = load_pdf('data/')

In [22]:
#Splitting the document as chunk
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [23]:
text_chunks  = text_split(extracted_data)
print('Length of my chunks',len(text_chunks))

Length of my chunks 4722


In [24]:
#Download HuggingFace embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embedding()

In [26]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [27]:
from pinecone import __file__
print(__file__)


c:\Users\praka\anaconda3\envs\geochatbot\lib\site-packages\pinecone\__init__.py


In [31]:
#Initializing Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="geochatbot"

#Creating Embeddings for Each of The Text Chunks & storing in pinecone db
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


In [ ]:
docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = 'What is infinite slopes'

docs = docsearch.similarity_search(query,k=3)
print(docs)

[Document(page_content='homogeneous. It is also assumed that it is possible to compute the seepage forces from the flow\nnet and the shearing strength of the soil from the Mohr-Coulomb theory.\nThe slope may be an ‘infinite’ one or a ‘finite’ one. An infinite slope represents the sur-\nface of a semi-infinite inclined soil mass; obviously, such a slope is rather hypothetical in\nnature. A slope of a finite extent, bounded by a top surface is said to be finite. Slopes involving', metadata={}), Document(page_content='homogeneous. It is also assumed that it is possible to compute the seepage forces from the flow\nnet and the shearing strength of the soil from the Mohr-Coulomb theory.\nThe slope may be an ‘infinite’ one or a ‘finite’ one. An infinite slope represents the sur-\nface of a semi-infinite inclined soil mass; obviously, such a slope is rather hypothetical in\nnature. A slope of a finite extent, bounded by a top surface is said to be finite. Slopes involving', metadata={}), Docum

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input = input(f'Input Promt:')
    result = qa({'query':user_input})
    print('Response : ', result['result'])